# [Gensim NLP trials](#gensim-nlp-trials)

In [ ]:
# !python -m spacy download en_core_web_sm

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import re
from itertools import combinations
from pathlib import Path
from time import time
from IPython.display import display

import altair as alt
import gensim.corpora as corpora
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
from gensim.models import nmf, Word2Vec
from nltk.corpus import stopwords
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.decomposition import NMF
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
%aimport src.pipe_helpers
from src.pipe_helpers import TextCleaner

%aimport src.gensim_helpers
from src.gensim_helpers import (
    compute_coherence_values,
    make_bigrams,
    remove_stopwords,
    lemmatization,
    sent_to_words,
    format_topics_sentences,
    get_bigrams_trigrams,
    plot_coherence_scores,
    compute_coherence_values,
)

%aimport src.visualization_helpers
from src.visualization_helpers import (
    altair_datetime_heatmap,
    plot_horiz_bar,
    plot_horiz_bar_gensim,
)

In [ ]:
SMALL_SIZE = 26
MEDIUM_SIZE = 28
BIGGER_SIZE = 30
plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["axes.facecolor"] = "white"
sns.set_style("darkgrid", {"legend.frameon": False})
sns.set_context("talk", font_scale=0.95, rc={"lines.linewidth": 2.5})

In [ ]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
%matplotlib inline

<a id="toc"></a>

## [Table of Contents](#table-of-contents)
0. [About](#about)
1. [User Inputs](#user-inputs)
2. [Load joined data](#load-joined-data)
3. [Topic modeling using Gensim NMF with TFIDF vectorization](#topic-modeling-using-gensim-nmf-with-tfidf-vectorization)
4. [Topic modeling using Gensim NMF without TFIDF vectorization](#topic-modeling-using-gensim-nmf-without-tfidf-vectorization)
   - 4.1. [Pre-processing for Gensim NMF](#pre-processing-for-gensim-nmf)
   - 4.2. [Gensim NMF](#gensim-nmf)
   - 4.3. [Exploring Gensim NMF topics combined with source data](#exploring-gensim-nmf-topics-combined-with-source-data)

<a id="about"></a>

## 0. [About](#about)

In this notebook, we will experiment with NLP models on the joined news listings data in `data/processed/*_processed.csv`

<a id="user-inputs"></a>

## 1. [User Inputs](#user-inputs)

We'll define below the variables that are to be used throughout the code.

In [ ]:
# Dataset
publication_name = "guardian"

# Data locations
data_dir_path = str(
    Path().cwd() / "data" / "processed" / f"{publication_name}_processed.csv"
)
cloud_run = True

# Custom stop words to include
manual_stop_words = ["nt", "ll", "ve"]

# Topic naming
gensim_non_tfidf_mapping_dict = {
    "guardian": {
        0: "Studying Comets and Meteors",  #
        1: "Rocket Launches - Testing",  #
        2: "Discover of Sub-Atomic particles",  #
        3: "Learning and Memory",  #
        4: "ISS",  #
        5: "Brain Research",  #
        6: "Academia",  ##
        7: "Rocket Launches - Moon Landing",  #
        8: "Pseudo space-science and Humanity - Opinion",  #
        9: "Imaging Stars - Astronomy",  #
        10: "Planetary Research",  #
        11: "Global Warming and Climate Science",  #
        12: "Dark Matter Theories",  ##
        13: "Space Funding Bodies",  ##
        14: "Mars Exploration",  #
    }
}

# General inputs
limit = 25
start = 10
step = 1
n_top_words = 10
random_state = 42

In [ ]:
def calculate_coherence(w2v_model, term_rankings):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for pair in combinations(term_rankings[topic_index], 2):
            pair_scores.append(w2v_model.wv.similarity(pair[0], pair[1]))
        # get the mean for all pairs in this topic
        topic_score = sum(pair_scores) / len(pair_scores)
        overall_coherence += topic_score
    # get the mean score across all topics
    return overall_coherence / len(term_rankings)


def get_descriptor(all_terms, H, topic_index, top):
    # reverse sort the values to sort the indices
    top_indices = np.argsort(H[topic_index, :])[::-1]
    # now get the terms corresponding to the top-ranked indices
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append(all_terms[term_index])
    return top_terms


class TokenGenerator:
    def __init__(self, documents, stopwords):
        self.documents = documents
        self.stopwords = stopwords
        self.tokenizer = re.compile(r"(?u)\b\w\w+\b")

    def __iter__(self):
        print("Building Word2Vec model ...")
        for doc in self.documents:
            tokens = []
            for tok in self.tokenizer.findall(doc):
                if tok in self.stopwords:
                    tokens.append("<stopword>")
                elif len(tok) >= 2:
                    tokens.append(tok)
            yield tokens


def fit_nmf_for_num_topics(
    start: int, limit: int, random_state: int, doc_term_matrix
) -> list:
    topic_models = []
    for num_topics in range(start, limit + 1):
        print(f" > Applying NMF with {num_topics:0d} topics...", end="")
        model = NMF(n_components=num_topics, max_iter=700, random_state=random_state)
        model_transformed = model.fit_transform(docs_terms)
        print("done")
        factors_dict = model.components_
        topic_models.append((num_topics, model_transformed, factors_dict))
    return topic_models


def compute_coherence_values_manually(
    topic_models_fitted: list, n_top_words: int, word2vec_model_fitted: Word2Vec
) -> None:
    k_values = []
    coherences = []
    for (k, W, H) in topic_models_fitted:
        # Get all of the topic descriptors - the term_rankings, based on top 10 terms
        term_rankings = []
        for topic_index in range(k):
            term_rankings.append(get_descriptor(terms, H, topic_index, n_top_words))
        # Now calculate the coherence based on our Word2vec model
        k_values.append(k)
        coherences.append(calculate_coherence(word2vec_model_fitted, term_rankings))
        print(f" > Applied NMF for k={k:0d} and found coherence={coherences[-1]:.4f}")


def print_top_words(
    topic_models: list,
    num_topics: int,
    n_top_words: int,
    start: int,
    feature_names: list,
    docs_terms,
    method: int = 2,
    random_state: int = 42,
) -> None:
    print(f"Top terms per topic, using random_state={random_state}:")
    if method == 1:
        # get the model that we generated earlier.
        W, H = topic_models[num_topics - start][1:]
        for topic_index in range(n_top_words):
            descriptor = get_descriptor(terms, H, topic_index, n_top_words)
            str_descriptor = " ".join(descriptor)
            print(f"Topic {topic_index+1:0d}: {str_descriptor}")
    else:
        best_model = NMF(
            n_components=num_topics, max_iter=700, random_state=random_state
        ).fit(docs_terms)
        for topic_idx, topic in enumerate(best_model.components_):
            message = f"Topic {topic_idx:0d}: "
            message += " ".join(
                [feature_names[i] for i in topic.argsort()[: -n_top_words - 1 : -1]]
            )
            print(message)

In [ ]:
# Get stop words from all packages
# NLTK
if not ((Path.cwd().parents[1]) / "nltk_data").exists():
    nltk.download("punkt")
    nltk.download("wordnet")
    nltk.download("stopwords")
    nltk.download("averaged_perceptron_tagger")
nltk_stop_words = set(stopwords.words("english"))
# Spacy and sklearn
spacy_stop_words = STOP_WORDS
sklearn_stop_words = stop_words.ENGLISH_STOP_WORDS

# Assemble manual list of stop words
spacy_not_in_sklearn = set(spacy_stop_words) - set(sklearn_stop_words)
nltk_not_in_sklearn = set(nltk_stop_words) - set(sklearn_stop_words)
all_stop_words = set(
    list(set(sklearn_stop_words))
    + list(spacy_not_in_sklearn)
    + list(nltk_not_in_sklearn)
)

# Manually add to stop words
for manual_stop_word in manual_stop_words:
    all_stop_words.add(manual_stop_word)

<a id="load-joined-data"></a>

## 2. [Load joined data](#load-joined-data)

We'll start by loading the joined data from from a publication, stored at `data/processed/<publication-name>_processed.csv`, into a `DataFrame`

In [ ]:
df = pd.read_csv(Path(data_dir_path))
df = df[["text", "year"]]
print(df.shape[0])
display(df.head())
# print(corpus[0])

In [ ]:
df["text"] = df["text"].str.lower()

In [ ]:
corpus_raw = df.loc[:, "text"].values.tolist()

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=None,
    lowercase=True,
    ngram_range=(1, 1),
    stop_words=all_stop_words,
    min_df=20,
    max_features=None,
    binary=False,
    strip_accents=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
)

In [ ]:
cell_st = time()

docs_terms = vectorizer.fit_transform(corpus_raw)
print(
    f"Created {docs_terms.shape[0]:0d} X {docs_terms.shape[1]:0d} "
    "TF-IDF-normalized document-term matrix"
)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

In [ ]:
terms = vectorizer.get_feature_names()
print(f"Vocabulary has {len(terms):0d} distinct terms")

In [ ]:
cell_st = time()

topic_models = fit_nmf_for_num_topics(start, limit, random_state, docs_terms)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

In [ ]:
cell_st = time()

docgen = TokenGenerator(corpus_raw, all_stop_words)
w2v_model = Word2Vec(docgen, size=500, min_count=20, sg=1, seed=1)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

In [ ]:
print(f"Model has {len(w2v_model.wv.vocab):0d} terms")

In [ ]:
cell_st = time()

compute_coherence_values_manually(topic_models, n_top_words, Word2Vec)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

In [ ]:
plot_coherence_scores(coherences, start, limit, step, (12, 4))

In [ ]:
best_n_topics = 15

In [ ]:
print_top_words(
    topic_models,
    best_n_topics,
    n_top_words,
    start,
    terms,
    docs_terms,
    method=2,
    random_state=random_state,
)

```
Topic 01: people, says, brain, like, life, time, human, world, think, species
Topic 02: rocket, spacex, space, launch, musk, company, falcon, flight, satellite, satellites
Topic 03: mars, beagle, martian, planet, rover, life, water, mission, lander, nasa
Topic 04: planets, planet, star, earth, stars, sun, kepler, life, solar, astronomers
Topic 05: comet, rosetta, philae, lander, comets, 67p, sun, dust, mission, surface
Topic 06: telescope, hubble, stars, galaxies, universe, galaxy, light, dark, astronomers, matter
Topic 07: moon, lunar, apollo, armstrong, space, nasa, china, astronauts, surface, earth
Topic 08: shuttle, nasa, columbia, discovery, launch, space, astronauts, mission, station, flight
Topic 09: asteroid, asteroids, earth, impact, rock, object, collision, hit, near, orbit
Topic 10: station, space, mir, russian, iss, crew, peake, astronaut, russia, international
Topic 11: cassini, saturn, titan, huygens, rings, moons, enceladus, spacecraft, probe, moon
Topic 12: science, uk, research, climate, space, government, said, scientific, scientists, public
Topic 13: higgs, particle, particles, lhc, matter, physics, dark, cern, energy, boson
Topic 14: black, hawking, gravitational, universe, waves, theory, holes, einstein, hole, relativity
Topic 15: pluto, horizons, planet, dwarf, solar, new, kuiper, stern, belt, charon
```

<a id="topic-modeling-using-gensim-nmf-without-tfidf-vectorization"></a>

## 4. [Topic modeling using Gensim NMF without TFIDF vectorization](#topic-modeling-using-gensim-nmf-without-tfidf-vectorization)

Here, we'll use Gensim's implementation of NMF, without TFIDF vectorization, to retrieve topics. This will be done without TFIDF Vectorization from either [`sklearn`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer) or [`gensim`](https://radimrehurek.com/gensim/models/tfidfmodel.html#gensim.models.tfidfmodel.TfidfModel) itself.

<a id="pre-processing-for-gensim-nmf"></a>

### 4.1. [Pre-processing for Gensim NMF](#pre-processing-for-gensim-nmf)

First, we'll clean the text of the articles

In [ ]:
cell_st = time()

pipe = Pipeline(steps=[("cleaner", TextCleaner(split=False))])
corpus_raw_cleaned = pipe.fit_transform(corpus_raw)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

We'll now tokenize the cleaned sentences into a list of words

In [ ]:
cell_st = time()

# data_words = list(sent_to_words(corpus_raw))
data_words = list(sent_to_words(corpus_raw_cleaned))

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

We'll use Gensim's `Phrases` module to build bigram and trigram models

In [ ]:
cell_st = time()

bigram_model, trigram_model = get_bigrams_trigrams(data_words, 5, 100)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

Next, we'll perform the following pre-processing
- remove stopwords
- (optional) create bigrams
- (optional) lemmatize

In [ ]:
cell_st = time()

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words, all_stop_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops, bigram_model)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(
    data_words_nostops, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]
)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

Now, we'll create a corpus comprising an assigned ID and corresponding frequency of words from the cleaned list of words (where stopwords were removed) above

In [ ]:
cell_st = time()

# Create Dictionary
# id2word = corpora.Dictionary(data_lemmatized)
id2word = corpora.Dictionary(data_words_nostops)

# Term Document Frequency for corpus
# corpus = [id2word.doc2bow(text) for text in data_lemmatized]
corpus = [id2word.doc2bow(text) for text in data_words_nostops]

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds")

<a id="gensim-nmf"></a>

### 4.2. [Gensim NMF](#gensim-nmf)

We'll now train Gensim's NMF model. A helper function below will iterate over the number of topics and compute the coherence score for each number.

In [ ]:
cell_st = time()

model_dict, coherence_values = compute_coherence_values(
    corpus=corpus,
    id2word=id2word,
    # texts=data_lemmatized,
    texts=data_words_nostops,
    limit=limit,
    start=start,
    step=step,
)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

The coherence scores are graphed below by number of topics used, [with an annotation](https://matplotlib.org/2.0.0/users/annotations.html#annotating-with-text-with-box) showing the number of topics with the highest coherence score

In [ ]:
plot_coherence_scores(coherence_values, start, limit, step, (12, 4))

**Observations**
1. There isn't strong evidence of an increasing trend in the number of topics, nor of a plateau in the scores. While Gensim model hyperparameter optimization could be further explored, the scores do not show a preference for a specific choice of number of topics over the others.

We'll use 15 topics for further exploration

In [ ]:
cell_st = time()

best_model = nmf.Nmf(
    corpus=corpus,
    id2word=id2word,
    num_topics=model_dict[15].num_topics,
    chunksize=len(corpus),  # no. of docs to be used in each training chunk
    passes=10,
    kappa=1.0,
    minimum_probability=0.01,
    w_max_iter=200,
    w_stop_condition=0.0001,
    h_max_iter=50,
    h_stop_condition=0.001,
    eval_every=10,
    normalize=True,
    random_state=random_state,
)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

Now, we'll print out all the topics found from the Gensim NMF model

In [ ]:
twords = {}
print(f"Top terms per topic, using random_state={random_state}:")
for topic, word in best_model.show_topics(
    num_topics=len(gensim_non_tfidf_mapping_dict[publication_name]), num_words=10
):
    words_cleaned = re.sub("[^A-Za-z ]+", "", word)
    twords[topic] = words_cleaned
    print(f"Topic {topic}:", words_cleaned.replace("  ", " "))

The topics and their top ten words are shown below, for the pre-determined choice of `random_state` of `42` specified in the Gensim NMF model

```
Top terms per cluster, using random_state=42:
Topic 0: life time earth comet years scientists like species book dna
Topic 1: space earth spacecraft satellites orbit satellite mission solar rocket moon
Topic 2: particles universe particle theory physics higgs energy lhc matter time
Topic 3: people memory like memories price day dont think remember things
Topic 4: space station nasa shuttle astronauts launch mission said flight crew
Topic 5: brain cells stem human tissue body cell neurons embryonic brains
Topic 6: number atomic species new matter xenon human tellurium dark numbers
Topic 7: says moon people like time lunar world going apollo think
Topic 8: said time dawkins think people human like new dont world
Topic 9: stars telescope star said light black astronomers galaxy years universe
Topic 10: planet planets solar earth water sun scientists surface pluto ice
Topic 11: work ice theory equation space climate surface time mathematical moduli
Topic 12: research says new university scientists work matter scientific uk dark
Topic 13: science scientific scientists research people public world climate new like
Topic 14: mars mission life surface martian planet water nasa said scientists
```

**Observations**
1. Three new topics are introduced here, relative to the `TFIDF+NMF` or CorEx approaches. Some existing ones are modified in their appearance.
2. One of these news topic names (`Dark Matter Theories`) is a weaker choice, based on the components words, and could be similar to the `Gravity and Black Holes` topic previously identified. Two topics could possibly be combined - `Brian Research` and `Learning and Memory` - resulting in 14 topics, as suggested by cross-validation with coherence-scores. NMF model hyper-parameter optimization could be explored to further study the topic selection.

In [ ]:
cell_st = time()

_ = plot_horiz_bar_gensim(
    best_model,
    id2word,
    gensim_non_tfidf_mapping_dict[publication_name],
    fig_size=(40, 35),
)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

We'll append the topic to the same row as each document in the original data

In [ ]:
cell_st = time()

df_with_topics = format_topics_sentences(
    best_model, corpus, df, gensim_non_tfidf_mapping_dict[publication_name]
)
display(df_with_topics.head(2))

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

<a id="exploring-gensim-nmf-topics-combined-with-source-data"></a>

### 4.3. [Exploring Gensim NMF topics combined with source data](#exploring-gensim-nmf-topics-combined-with-source-data)

Here, we will show a heatmap of the most popular topic by year, found by Gensim's implementation of NMF (recall this was done above without TFIDF Vectorization)

In [ ]:
topics_by_timeframe = (
    df_with_topics.groupby(["most_popular_topic", "year"])
    .size()
    .reset_index()
    .sort_values(by=["most_popular_topic", 0, "year"], ascending=False)
    .rename(columns={0: "count"})
)
topics_by_timeframe.head()

In [ ]:
altair_datetime_heatmap(
    topics_by_timeframe,
    x="year:O",
    y="most_popular_topic:N",
    xtitle="Year",
    ytitle="Most popular topic",
    tooltip=[
        {"title": "Year", "field": "year", "type": "ordinal",},
        {
            "title": "Most popular topic",
            "field": "most_popular_topic",
            "type": "nominal",
        },
        {
            "title": "Number of occurrences as main topic",
            "field": "count",
            "type": "quantitative",
        },
    ],
    cmap="yelloworangered",
    legend_title="",
    color_by_col="count:Q",
    yscale="log",
    axis_tick_font_size=12,
    axis_title_font_size=16,
    title_font_size=20,
    legend_fig_padding=10,  # default is 18
    y_axis_title_alignment="left",
    fwidth=700,
    fheight=450,
    file_path=Path().cwd() / "reports" / "figures" / "my_heatmap.html",
    save_to_html=False,
    sort_y=[],
    sort_x=[],
)

Next, we will show a bar chart of the number of occurrences of the `"Space Funding Bodies"` as the most popular topic, relative to the year 1980
- this will approximate the public interest in changes in this topic over the years investigated

In [ ]:
funds = (
    topics_by_timeframe[
        topics_by_timeframe["most_popular_topic"] == "Space Funding Bodies"
    ]
    .set_index("year")["count"]
    .sort_index()
)
funds / funds.loc[funds.index.min()]
funds = funds / funds.loc[funds.index.min()]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
funds.plot(kind="bar", ax=ax, rot=45, align="edge", width=0.8)
ax.set_title(
    "Cyclic variation in funding as main topic in article",
    fontsize=18,
    fontweight="bold",
)
ax.set_xlabel(None)
h = plt.ylabel("Funding\n(rel. to 1981)", labelpad=65, fontweight="bold")
h.set_rotation(0)

**Observations**
1. The multi-modal aspects of the `TFIDF+NMF` and CorEx implementations showing a broadened peak centered at 2014, a weaker peak in articles published under this topic in 2004 and a peak in 2007also appear here. The choice of this topic was not as difficult here as it was for two of the other assigned topic names.
2. Peak heights are distinctly weaker than those found in `TFIDF+NMF` and CorEx spproaches indicating documents previously assigned to this topic are now being placed in another topic.

<a id="conclusions"></a>

## 5. [Conclusions](#conclusions)

1. Without `TFIDF` vectorization, `NMF` found three new topics that were not observed previously. The number of topics has some evidence of being one too many. If such an approach (`NMF` without `TFIDF` vectorization) is to be used further, then the performance of model hyperparameter optimization with 14 topics should be investigated.